In [3]:
pip install fastapi

  Using cached fastapi-0.112.0-py3-none-any.whl.metadata (27 kB)
  Using cached starlette-0.37.2-py3-none-any.whl.metadata (5.9 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.20.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
Using cached fastapi-0.112.0-py3-none-any.whl (93 kB)
Using cached pydantic-2.8.2-py3-none-any.whl (423 kB)
Using cached pydantic_core-2.20.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
Using cached starlette-0.37.2-py3-none-any.whl (71 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
from typing import Union

from fastapi import FastAPI, HTTPException
import pandas as pd
import os
import requests

import json
import time
from tqdm import tqdm


app = FastAPI()

In [6]:
def get_key():
    key = os.getenv('MOVIE_API_KEY')
    return key

def req(movie_cd, url_param = {}):
    url = gen_url(movie_cd, url_param)
    r = requests.get(url)
    code = r.status_code
    data = r.json()
    print(data)
    return code, data

def gen_url(movie_cd, url_param = {}):
    #base_url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json"
    #base_url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json"
    base_url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/company/searchCompanyList.json"
    
    key = get_key()
    url = f"{base_url}?key={key}&curPage=1"
    for k, v in url_param.items():
        url = url + f"&{k}={v}"
    return url

#df = pd.read_parquet('/home/kim1/code/ffapi/data')

In [7]:
@app.get("/")
def read_root():
    return {"Hello": "World"}

In [8]:
@app.get("/sample")
def sample_data():
    df = pd.read_parquet('/home/kim1/code/ffapi/data')
    sample_df = df.sample(n=5)
    r = sample_df.to_dict(orient='records')
    return r

In [9]:
@app.get("/movie/{movie_cd}")
def movie_meta(movie_cd: str):
    df = pd.read_parquet('/home/kim1/code/ffapi/data')

    # df 에서 movieCd == movie_cd row 를 조회 df[['a'] === b] ...
    meta_df = df[df['movieCd'] == movie_cd]

    if meta_df.empty:
        raise HTTPException(status_code=404, detail="영화를 찾을 수 없습니다")

    # 조회된 데이터를 .to_dict() 로 만들어 아래에서 return
    r = meta_df.iloc[0].to_dict()

    if r['repNationCd'] is None:
        _,rr = req(movie_cd)
        print(rr)
        new_rr=rr["movieInfoResult"]["movieInfo"]

        nation = new_rr["nations"][0]

        if nation['nationNm'] != '한국':
            r['repNationCd'] = 'F'
        else:
            r['repNationCd'] = 'N'

    return r